In [17]:
from uniswap_universal_router_decoder import FunctionRecipient, RouterCodec
from web3 import Account, Web3
from eth_account.messages import encode_defunct

import requests
import json

In [31]:
# setup static params

# metamask wallet
metamask_wallet_account = ""  # TODO:
private_key = "" 
account = Account.from_key(private_key)

chain_id = 314
rpc_endpoint = "https://api.node.glif.io/rpc/v1"
w3 = Web3(Web3.HTTPProvider(rpc_endpoint))

wpfil_address = Web3.to_checksum_address('0x57E3BB9F790185Cfe70Cc2C15Ed5d6B84dCf4aDb')
with open('abis/wpfilABI.json', 'r') as fp:
    wpfil_abi = json.load(fp)
wpfil_contract = w3.eth.contract(address=wpfil_address, abi=wpfil_abi)

# router for executing swaps
ur_address = Web3.to_checksum_address("0x83702C6356A1028A900F83d446D189a31646a16b")
ur_abi = '[{"inputs":[{"components":[{"internalType":"address","name":"permit2","type":"address"},{"internalType":"address","name":"weth9","type":"address"},{"internalType":"address","name":"seaport","type":"address"},{"internalType":"address","name":"nftxZap","type":"address"},{"internalType":"address","name":"x2y2","type":"address"},{"internalType":"address","name":"foundation","type":"address"},{"internalType":"address","name":"sudoswap","type":"address"},{"internalType":"address","name":"nft20Zap","type":"address"},{"internalType":"address","name":"cryptopunks","type":"address"},{"internalType":"address","name":"looksRare","type":"address"},{"internalType":"address","name":"routerRewardsDistributor","type":"address"},{"internalType":"address","name":"looksRareRewardsDistributor","type":"address"},{"internalType":"address","name":"looksRareToken","type":"address"},{"internalType":"address","name":"v2Factory","type":"address"},{"internalType":"address","name":"v3Factory","type":"address"},{"internalType":"bytes32","name":"pairInitCodeHash","type":"bytes32"},{"internalType":"bytes32","name":"poolInitCodeHash","type":"bytes32"}],"internalType":"struct RouterParameters","name":"params","type":"tuple"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"ContractLocked","type":"error"},{"inputs":[],"name":"ETHNotAccepted","type":"error"},{"inputs":[{"internalType":"uint256","name":"commandIndex","type":"uint256"},{"internalType":"bytes","name":"message","type":"bytes"}],"name":"ExecutionFailed","type":"error"},{"inputs":[],"name":"FromAddressIsNotOwner","type":"error"},{"inputs":[],"name":"InsufficientETH","type":"error"},{"inputs":[],"name":"InsufficientToken","type":"error"},{"inputs":[],"name":"InvalidBips","type":"error"},{"inputs":[{"internalType":"uint256","name":"commandType","type":"uint256"}],"name":"InvalidCommandType","type":"error"},{"inputs":[],"name":"InvalidOwnerERC1155","type":"error"},{"inputs":[],"name":"InvalidOwnerERC721","type":"error"},{"inputs":[],"name":"InvalidPath","type":"error"},{"inputs":[],"name":"InvalidReserves","type":"error"},{"inputs":[],"name":"LengthMismatch","type":"error"},{"inputs":[],"name":"NoSlice","type":"error"},{"inputs":[],"name":"SliceOutOfBounds","type":"error"},{"inputs":[],"name":"SliceOverflow","type":"error"},{"inputs":[],"name":"ToAddressOutOfBounds","type":"error"},{"inputs":[],"name":"ToAddressOverflow","type":"error"},{"inputs":[],"name":"ToUint24OutOfBounds","type":"error"},{"inputs":[],"name":"ToUint24Overflow","type":"error"},{"inputs":[],"name":"TransactionDeadlinePassed","type":"error"},{"inputs":[],"name":"UnableToClaim","type":"error"},{"inputs":[],"name":"UnsafeCast","type":"error"},{"inputs":[],"name":"V2InvalidPath","type":"error"},{"inputs":[],"name":"V2TooLittleReceived","type":"error"},{"inputs":[],"name":"V2TooMuchRequested","type":"error"},{"inputs":[],"name":"V3InvalidAmountOut","type":"error"},{"inputs":[],"name":"V3InvalidCaller","type":"error"},{"inputs":[],"name":"V3InvalidSwap","type":"error"},{"inputs":[],"name":"V3TooLittleReceived","type":"error"},{"inputs":[],"name":"V3TooMuchRequested","type":"error"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"RewardsSent","type":"event"},{"inputs":[{"internalType":"bytes","name":"looksRareClaim","type":"bytes"}],"name":"collectRewards","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes","name":"commands","type":"bytes"},{"internalType":"bytes[]","name":"inputs","type":"bytes[]"}],"name":"execute","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"bytes","name":"commands","type":"bytes"},{"internalType":"bytes[]","name":"inputs","type":"bytes[]"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"execute","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256[]","name":"","type":"uint256[]"},{"internalType":"uint256[]","name":"","type":"uint256[]"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC1155BatchReceived","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC1155Received","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"bytes","name":"","type":"bytes"}],"name":"onERC721Received","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"bytes4","name":"interfaceId","type":"bytes4"}],"name":"supportsInterface","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"int256","name":"amount0Delta","type":"int256"},{"internalType":"int256","name":"amount1Delta","type":"int256"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"uniswapV3SwapCallback","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'

In [32]:
# send a request to get the data needed to execute the swap to OKU Trade's API

# command line looks like the following
"""
curl 'https://oku-canoe.fly.dev/market/usor/swap_quote' -X POST -H 'content-type: application/json'  \
    --data-raw '{"chain":"filecoin",
                 "account":"0x0000000000000000000000000000000000000000",  # metamask wallet
                 "inTokenAddress":"0x06eFdBFf2a14a7c8E15944D1F4A48F9F95F663A4",
                 "outTokenAddress":"0x5300000000000000000000000000000000000004",
                 "isExactIn":true,"gasPrice":100000000,"slippage":100,"inTokenAmount":"1"}'
"""
# lets do this with json-rpc
url = 'https://oku-canoe.fly.dev/market/usor/swap_quote'
payload = {
    "chain": "filecoin",
    "account": metamask_wallet_account,
    "inTokenAddress": "0x60E1773636CF5E4A227d9AC24F20fEca034ee25A",
    "outTokenAddress": "0x57E3BB9F790185Cfe70Cc2C15Ed5d6B84dCf4aDb",
    "isExactIn": True,
    "gasPrice": 100000000,
    "slippage": 100,
    "inTokenAmount": "1"
}

headers = {
    'content-type': 'application/json'
}
response = requests.post(url, data=json.dumps(payload), headers=headers)

In [33]:
coupon = response.json()['coupon']
signature_request = response.json()['signingRequest']

In [48]:
permit2Address = signature_request['permit2Address']
# sign the permitSignature
print(permit2Address)
print(signature_request['permitSignature'][0])

0xb81363578d377F8DA3902e9e705FD60198a9cEc2
{'permit': {'details': {'token': '0x60E1773636CF5E4A227d9AC24F20fEca034ee25A', 'amount': '1000000000000000000', 'expiration': '1704135102', 'nonce': '1'}, 'spender': '0x83702C6356A1028A900F83d446D189a31646a16b', 'sigDeadline': '1704135102'}, 'signature': '0x8d4cb1b0892d8e5bd395eb4b557d01c9e7bf6f53e0c7da6f413d9e8a6ac532c24ad10b8c2f09851268277bd642fbcd655b98c685144048703a25d9ed4af208041c'}


In [35]:
message_hash = Web3.keccak(text=json.dumps(signature_request['permitSignature'][0])).hex()

In [36]:
signed_message = w3.eth.account.sign_message(encode_defunct(hexstr=message_hash), private_key=private_key)

In [37]:
signed_message

SignedMessage(messageHash=HexBytes('0x7c3fa1a56dab6b808a5e7a783d5f618a7a642ea0b53a6a9c446dce738a105af8'), r=63911618395664237964458822197177947534347848995413599528578388706233638269634, s=33840501530256337093777862609278627474580769183592314167092887398496777406468, v=28, signature=HexBytes('0x8d4cb1b0892d8e5bd395eb4b557d01c9e7bf6f53e0c7da6f413d9e8a6ac532c24ad10b8c2f09851268277bd642fbcd655b98c685144048703a25d9ed4af208041c'))

In [38]:
signature_request['permitSignature'][0]['signature'] = signed_message['signature'].hex()

In [39]:
signature_request['permitSignature'][0]

{'permit': {'details': {'token': '0x60E1773636CF5E4A227d9AC24F20fEca034ee25A',
   'amount': '1000000000000000000',
   'expiration': '1704135102',
   'nonce': '1'},
  'spender': '0x83702C6356A1028A900F83d446D189a31646a16b',
  'sigDeadline': '1704135102'},
 'signature': '0x8d4cb1b0892d8e5bd395eb4b557d01c9e7bf6f53e0c7da6f413d9e8a6ac532c24ad10b8c2f09851268277bd642fbcd655b98c685144048703a25d9ed4af208041c'}

In [40]:
# now send the execution request
url = 'https://oku-canoe.fly.dev/market/usor/execution_information'
payload = {
    "coupon": coupon,
    "signingRequest": signature_request['permitSignature'],
}
headers = {
    'content-type': 'application/json'
}
exec_info = requests.post(url, data=json.dumps(payload), headers=headers)

In [44]:
exec_info.json()

{'trade': {'chainId': 314,
  'to': '0x83702C6356A1028A900F83d446D189a31646a16b',
  'data': '0x3593564c000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000659309be0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000010000000000000000000000000087c2067f301bb48f377328cdda689578376b9c7d0000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000dc0cd3f0b379fe500000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000002b60

In [45]:
amount_in = 0.05 * 10**18

trx_params = {
        "from": account.address,
        "to": ur_address,
        "gas": 150000000,  # in units of GAS
        "maxPriorityFeePerGas": 200000,  # in units of attofil
        "maxFeePerGas": 200000,  # in units of attofil, so 100*10**9 = 100 nanoFIL
        # "type": '0x2',
        "chainId": chain_id,
        "value": int(amount_in),
        "nonce": w3.eth.get_transaction_count(account.address),
        "data": exec_info.json()['trade']['data'],
}

In [46]:
raw_transaction = w3.eth.account.sign_transaction(trx_params, account.key).rawTransaction

In [47]:
trx_hash = w3.eth.send_raw_transaction(raw_transaction)
print(f"Trx Hash: {trx_hash}")

Trx Hash: b'D\x01\xce\xef\x17\x81\x0e\xf4&\xf0O&l5\xe2\t\xb2@\xf6\x80u\xe8\xbf\x90?\xc9\xf4Aj\xe8\xfa\xbb'
